# Restaurant survival data preparation
In this notebook, data scraped from TripAdvisor in 2021 and data scraped from TripAdvisor in 2023 is joined into a single dataset. Some features are also calculated in this notebook, as their calculation takes time.

This data will be used in another notebook to see which features influence a restaurant's survival chances.

In [1]:
import numpy as np
import pandas as pd
import gc

import geopy.distance

In [2]:
# Load the 2023 data and join it into a single file.

df_ni_new = pd.read_csv('/kaggle/input/restraunt-survival/NI_with_webscan.csv')
df_sk_new = pd.read_csv('/kaggle/input/restraunt-survival/SK_with_webscan.csv')
df_bg_new = pd.read_csv('/kaggle/input/restraunt-survival/BG_with_webscan.csv')
df_fi_new = pd.read_csv('/kaggle/input/restraunt-survival/FI_with_webscan.csv')

df_new = pd.concat([df_ni_new, df_sk_new, df_bg_new, df_fi_new])
del df_ni_new, df_sk_new, df_bg_new, df_fi_new
gc.collect()

10

In [3]:
# Load the 2021 data and select rows for relevant countries.

df_old = pd.read_csv('/kaggle/input/tripadvisor-european-restaurants/tripadvisor_european_restaurants.csv')
df_ni_old = df_old.loc[df_old['country'] == 'Northern Ireland']
df_sk_old = df_old.loc[df_old['country'] == 'Slovakia']
df_bg_old = df_old.loc[df_old['country'] == 'Bulgaria']
df_fi_old = df_old.loc[df_old['country'] == 'Finland']

del df_old
gc.collect()

/tmp/ipykernel_20/2694908195.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_old = pd.read_csv('/kaggle/input/tripadvisor-european-restaurants/tripadvisor_european_restaurants.csv')


0

In [4]:
# Clean the province column.

df_ni_old['province'] = df_ni_old['province'].fillna(df_ni_old['region'])
del df_ni_old['region']
df_ni_old.loc[df_ni_old['province'] == 'Dunmurry', 'province'] = 'Belfast'

df_sk_old['province'] = df_sk_old['region']
del df_sk_old['region']

df_bg_old['province'] = df_bg_old['province'].fillna(df_bg_old['region'])
del df_bg_old['region']
df_bg_old.loc[df_bg_old['province'] == 'Sofia', 'province'] = 'Sofia Region'

df_fi_old.loc[df_fi_old['city'] == 'Halikko', 'region'] = 'Southwest Finland'
df_fi_old.loc[df_fi_old['city'] == 'Kronoby', 'region'] = 'Ostrobothnia'
df_fi_old.loc[df_fi_old['restaurant_link'] == 'g189896-d21010741', 'region'] = 'Southwest Finland'
df_fi_old.loc[df_fi_old['restaurant_link'] == 'g13297777-d13810040', 'region'] = 'Southwest Finland'
df_fi_old.loc[df_fi_old['restaurant_link'] == 'g189940-d13116297', 'region'] = 'Southern Ostrobothnia'
df_fi_old.loc[df_fi_old['restaurant_link'] == 'g189940-d12720762', 'region'] = 'Southwest Finland'
df_fi_old['province'] = df_fi_old['region']
del df_fi_old['region']

In [5]:
# Join the 2021 data into a single dataframe.

df_old = pd.concat([df_ni_old, df_sk_old, df_bg_old, df_fi_old])
del df_ni_old, df_sk_old, df_bg_old, df_fi_old
gc.collect()

0

In [6]:
# Some more cleaning for 2021 data.

df_old['price_level'] = df_old['price_level'].fillna('No data')

df_old['total_reviews_count'] = df_old['total_reviews_count'].fillna(df_old['reviews_count_in_default_language'])
df_old['total_reviews_count'] = df_old['total_reviews_count'].fillna(0)

In [7]:
# Remove entries that turned out to be duplicates.
df_new = df_new.loc[df_new['status'] != 'duplicate']

In [8]:
# Estimate year each restaurant was reported on TripAdvisor.
df_new['last_review_year'] = df_new['last_review'].apply(lambda x: str(x)[-4:] if str(x)!='nan' else 0).astype(np.int16)
df_new['inc_number'] = df_new['id'].apply(lambda x: x.split('-d')[1]).astype(np.int32)
df_inc = df_new.groupby('last_review_year').agg({'inc_number':np.max}).reset_index()
df_inc = df_inc.loc[df_inc['last_review_year'] > 2000]
df_inc = df_inc.loc[df_inc['last_review_year'] < 2021]
df_inc = df_inc.sort_values('last_review_year', ascending=False)
df_inc = df_inc.reset_index(drop=True)
df_new['est_report_year'] = 2021

for i in range(len(df_inc)):
    df_new.loc[df_new['inc_number'] <= df_inc['inc_number'].iloc[i], 'est_report_year'] = df_inc['last_review_year'].iloc[i]

In [9]:
# Join the 2021 and 2023 data into a single dataframe.

df_new = df_new.rename(columns={'is_claimed':'claimed_2023', 'rating':'rating_2023'})

df_join = df_new[['id', 'status', 'claimed_2023', 'rating_2023', 'last_review', 'n_reviews', 'last_review_year', 'est_report_year',
                  'fb_last_update_year', 'website_type', 'website_status', 'fb_rating', 'fb_reviews', 'English', 'non-English',
                  'local']]

df_old = df_old.merge(df_join, left_on='restaurant_link', right_on='id', how='inner')
del df_old['id']

In [10]:
df_old['correct_coordinates'] = 1
df_old.loc[df_old['latitude'].isnull(), 'correct_coordinates'] = 0
df_old.loc[df_old['longitude'].isnull(), 'correct_coordinates'] = 0
df_old.loc[(df_old['country'] == 'Northern Ireland') & ((df_old['latitude'] < 54) | (df_old['longitude'] > -5.4)),
           'correct_coordinates'] = 0
df_old.loc[(df_old['country'] == 'Slovakia') & ((df_old['latitude'] < 47) | (df_old['latitude'] > 50)),
          'correct_coordinates'] = 0
df_old.loc[(df_old['country'] == 'Bulgaria') & (df_old['longitude'] < 22), 'correct_coordinates'] = 0
df_old.loc[(df_old['country'] == 'Finland') & (df_old['latitude'] < 45), 'correct_coordinates'] = 0

In [11]:
# Calculate a number of distance features that count the number of restaurants meeting certain criteria
# and located within some distance from the target restaurant.

country_list = ['Northern Ireland', 'Slovakia', 'Bulgaria', 'Finland']

def proximate_restaurants(df_all, df_filtered, lat_diff, long_diff, country_list, max_dist):
    all_close = []
    for country in country_list:
        df_i = df_all.loc[df_all['country'] == country]
        df_filtered_i = df_filtered.loc[df_filtered['latitude'].notnull() & (df_filtered['country'] == country)]
      
        for i in range(len(df_i)):
            if df_i['correct_coordinates'].iloc[i] == 1:
                lat_i = df_i['latitude'].iloc[i]
                long_i = df_i['longitude'].iloc[i]
                coords_i = (lat_i, long_i)
                n = 0
                df_j = df_filtered_i.loc[(df_filtered_i['latitude'] - lat_i < lat_diff) &
                                         (df_filtered_i['latitude'] - lat_i > (-1 * lat_diff)) &
                                         (df_filtered_i['longitude'] - long_i < long_diff) &
                                         (df_filtered_i['longitude'] - long_i > (-1 * long_diff))]
                df_j = df_j.reset_index(drop=True)
                for j in range(len(df_j)):
                    coords_j = (df_j['latitude'].iloc[j], df_j['longitude'].iloc[j])
                    dist = geopy.distance.geodesic(coords_i, coords_j).km
                    if dist < max_dist:
                        n += 1
                all_close.append(n)
            else:
                all_close.append(0)
            if str(i).endswith('00'):
                print(str(i))
        print(country)
    return all_close

In [12]:
df_old['all_1km'] = proximate_restaurants(df_old, df_old, 0.02, 0.1, country_list, 1)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
Northern Ireland
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
Slovakia
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
Bulgaria
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
Finland


In [13]:
df_old['all_200m'] = proximate_restaurants(df_old, df_old, 0.01, 0.05, country_list, 0.2)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
Northern Ireland
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
Slovakia
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
Bulgaria
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
Finland


In [14]:
df_old['plus_10_1km'] = proximate_restaurants(df_old, df_old.loc[df_old['total_reviews_count'] > 10], 0.02, 0.1, country_list, 1)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
Northern Ireland
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
Slovakia
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
Bulgaria
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
Finland


In [15]:
df_old['plus_10_200m'] = proximate_restaurants(df_old, df_old.loc[df_old['total_reviews_count'] > 10], 0.01, 0.05, country_list, 0.2)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
Northern Ireland
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
Slovakia
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
Bulgaria
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
Finland


In [16]:
df_old['new_1km'] = proximate_restaurants(df_old, df_old.loc[df_old['est_report_year'] > 2019], 0.02, 0.1, country_list, 1)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
Northern Ireland
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
Slovakia
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
Bulgaria
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
Finland


In [17]:
df_old['new_1km_2021'] = proximate_restaurants(df_old, df_old.loc[df_old['est_report_year'] > 2020], 0.02, 0.1, country_list, 1)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
Northern Ireland
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
Slovakia
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
Bulgaria
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
Finland


In [18]:
# Save the result.
df_old.to_csv('TA_restaurants_joined.csv', index=False)